In [1]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = "poshmark_listings.db"
conn = sqlite3.connect(db_path)

# # List all tables
# tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
# print("Available tables:")
# display(tables)

In [6]:
import os
import csv

def count_rows_in_csvs(folder_path, skip_header=True):
    total_rows = 0

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
                reader = csv.reader(csvfile)
                row_count = sum(1 for row in reader)
                if skip_header:
                    row_count = max(0, row_count - 1)  # avoid negative counts
                print(f"{filename}: {row_count} rows")
                total_rows += row_count

    print(f"\nTotal rows across all CSV files: {total_rows}")
    return total_rows

# Example usage
folder = "seller_items"
count_rows_in_csvs(folder)


items_flowers4shiba_2025-05-11_06-33-29.csv: 203 rows
items_alyssajordyn_2025-05-11_01-45-37.csv: 21 rows
items_brianwhite2310_2025-05-11_03-21-22.csv: 6 rows
items_glamourkitty_2025-05-11_06-53-21.csv: 9 rows
items_ihaveklcstyle_2025-05-11_07-37-57.csv: 47 rows
items_bobby4776_2025-05-11_03-10-38.csv: 14 rows
items_furrysheep_2025-05-11_06-41-23.csv: 1 rows
items_desyredenise_2025-05-11_05-22-42.csv: 11 rows
items_codywaltman_2025-05-11_04-36-03.csv: 19 rows
items_garsantiques_2025-05-11_06-44-34.csv: 14 rows
items_christopherg407_2025-05-11_04-22-03.csv: 8 rows
items_apidde71_2025-05-11_02-11-43.csv: 8 rows
items_billups73_2025-05-11_02-58-59.csv: 3 rows
items_jonellecannon_2025-05-11_08-47-07.csv: 4 rows
items_josephsolazy_2025-05-11_08-49-27.csv: 24 rows
items_edithgc_2025-05-11_05-53-16.csv: 4 rows
items_jovialgecko_2025-05-11_08-50-22.csv: 9 rows
items_garciaaudrey61_2025-05-11_06-43-49.csv: 47 rows
items_isaiah_w55_2025-05-11_07-46-14.csv: 1 rows
items_ayeleets_2025-05-11_02-30-

3808572

In [5]:
query = """
SELECT COUNT(*)
FROM listings
"""

result = pd.read_sql(query, conn)
display(result)

,COUNT(*)
0,116830


In [2]:
query = """
SELECT 
    Seller,
    COUNT(*) AS item_count,
    AVG(CAST(REPLACE(REPLACE(Price, '$', ''), ',', '') AS REAL)) AS avg_price
FROM listings
WHERE Price NOT NULL AND Price != 'N/A'
GROUP BY Seller
ORDER BY avg_price ASC
LIMIT 100;
"""

result = pd.read_sql(query, conn)
display(result)

,Seller,item_count,avg_price
0,a1beltran,2,3.000000
1,a_smith310,1,3.000000
2,aaalmaloweee,2,3.000000
3,aaron29_theking,1,3.000000
4,0brooke1,38,4.315789
...,...,...,...
95,123alphabet,9,6.333333
96,1broketeach,144,6.354167
97,_sofia_,35,6.400000
98,aarynsmith77,10,6.400000


In [7]:
query = """
SELECT Seller, COUNT(*) AS item_count
FROM listings
WHERE CategoryName = 'Men > Shoes'
  AND Size IN ('8', '8.5', '9')
GROUP BY Seller
HAVING item_count > 10
ORDER BY item_count DESC;
"""

result = pd.read_sql(query, conn)
display(result)

,Seller,item_count
0,cjsplash,41
1,alfredoflores21,35
2,jredd2007,29
3,2wears,21
4,drbcollectibles,15


In [10]:
query = """
SELECT
    Seller,
    COUNT(*) AS item_count,
    ROUND(AVG(CAST(REPLACE(REPLACE(Price, '$', ''), ',', '') AS REAL)), 2) AS avg_price
FROM listings
WHERE CategoryName LIKE 'Men >%'
  AND Price NOT NULL
  AND Price != 'N/A'
  AND CAST(REPLACE(REPLACE(Price, '$', ''), ',', '') AS REAL) < 10
GROUP BY Seller
HAVING item_count > 20
ORDER BY item_count DESC;
"""

result = pd.read_sql(query, conn)
display(result)

,Seller,item_count,avg_price
0,gbresaleking,483,7.77
1,dhmunoz,131,7.47
2,jc_ramos510,102,7.38
3,taysallee,92,6.92
4,jredd2007,88,8.50
5,radocat,83,7.45
6,cityguymarket,72,7.83
7,cleoraines,70,6.96
8,amedom,59,7.63
9,anasaldierna,58,5.00


In [6]:
query = """
SELECT DISTINCT CategoryName
FROM listings
WHERE CategoryName IS NOT NULL AND CategoryName != 'N/A'
ORDER BY CategoryName;
"""

result = pd.read_sql(query, conn)
with pd.option_context(
    "display.max_rows", None,
    "display.max_columns", None,
    "display.max_colwidth", None
):
    display(result)

,CategoryName
0,"Electronics > Cameras, Photo & Video"
1,"Electronics > Car Audio, Video & GPS"
2,Electronics > Cell Phones & Accessories
3,"Electronics > Computers, Laptops & Parts"
4,Electronics > Headphones
5,Electronics > Media
6,Electronics > Networking
7,Electronics > Other
8,Electronics > Portable Audio & Video
9,Electronics > Tablets & Accessories


In [6]:
# Close connection
conn.close()